In [ ]:
%%capture
!pip install bitsandbytes accelerate transformers optimum

In [ ]:
from google.colab import drive

drive.mount("/content/drive")
DRIVE_ROOT = "/content/drive/MyDrive"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from huggingface_hub import login

login("****************************")

- `AutoModelForCausalLM`: Hugging Face의 사전 훈련된 LLM을 로드하는 클래스
- `AutoTokenizer`: 해당 모델에 맞는 토크나이저를 자동으로 불러오는 클래스
- `BitsAndBytesConfig`: 양자화 설정을 위한 클래스. 메모리 절약을 위해 bitsandbytes를 활용.
- `AutoConfig`: 모델의 구성을 자동으로 가져오는 클래스

In [ ]:
import torch
import bitsandbytes
import accelerate
import transformers
import optimum
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoConfig

In [ ]:
MODEL_NAME = "google/gemma-7b-it"

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True,  # 모델을 4비트 양자화, VRAM(그래픽 메모리) 절약
    bnb_4bit_quant_type="nf4",  # Normal Float 4 사용, fp4보다 연산 정밀도 높음
    bnb_4bit_compute_dtype=torch.bfloat16,  # 연산 시 bfloat16 사용 (속도, 메모리 절약)
    bnb_4bit_use_double_quant=True,  # 이중 양자화 해 추가적 메모리 절약
)

# 사전 학습된 모델의 토크나이저 로드. 최대 입력 토큰 길이를 4096으로
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, model_max_length=4096)
# 패딩을 왼쪽에 추가. 일반적으로 GPT 계열 모델은 왼쪽 패딩을 사용
tokenizer.padding_side = "left"
# 패딩 토큰을 EOS(문장 종료) 토큰과 동일하게 설정
# 일부 모델은 pad_token이 없으므로 EOS를 패딩 토큰으로 사용해 오류 방지
tokenizer.pad_token = tokenizer.eos_token

# 모델 로드
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map = "auto",
    trust_remote_code = True,
    quantization_config=quantization_config,
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
import pandas as pd
from string import Template
from pathlib import Path
import numpy as np
import os
import warnings
warnings.simplefilter("ignore")
import torch
from transformers import pipeline, AutoTokenizer
from tqdm.notebook import tqdm

In [ ]:
is_rerun = False  # os.getenv("KAGGLE_IS_COMPETITION_RERUN")

In [ ]:
data_path = Path(os.path.join(DRIVE_ROOT, "Colab Notebooks", "input", "LLM Prompt Recovery"))

if is_rerun:
    test = pd.read_csv(data_path / "test.csv", index_col="id")
    test["rewrite_prompt"] = "-"
else:
    test = pd.read_csv(data_path / "train.csv", index_col="id")

test.head()

,original_text,rewrite_prompt,rewritten_text
id,,,
-1,The competition dataset comprises text passage...,"Convert this into a sea shanty: """"""The competi...",Here is your shanty: (Verse 1) The text is rew...


In [ ]:
from torch import nn

In [ ]:
# PyTorch 기반 perplexity 계산을 위한 클래스
class Perplexity(nn.Module):
    def __init__(self, reduce: bool = True):
        super().__init__()
        self.loss_fn = nn.CrossEntropyLoss()
        self.reduce = reduce

    def forward(self, logits, labels):
        """
        Args:
            logits: 모델이 예측한 확률 값 또는 로짓 값 [batch_size, seq_len, vocab_size]
            labels: 정답 레이블 [batch_size, seq_len]
        Returns:
            perplexity: reduce가 True이면 평균 perplexity 반환, False이면 미니배치별 개별 perplexity 반환
        """
        # 각 시퀀스에서 마지막 단어를 제외한 로짓 값들만 남긴다 [batch_size, seq_len - 1, vocab_size]
        shift_logits = logits[..., :-1, :].contiguous()
        # 첫 번째 단어를 제외한 정답 레이블 [batch_size, seq_len - 1]
        shift_labels = labels[..., 1:].contiguous()

        perplexity = []
        for i in range(labels.shape[0]):
            perplexity.append(self.loss_fn(shift_logits[i], shift_labels[i]))
        perplexity = torch.stack(perplexity, dim=0)
        # perplexity = torch.exp(perplexity)
        if self.reduce:
            perplexity = torch.mean(perplexity)
        return perplexity

In [ ]:
perp = Perplexity()

In [ ]:
def format_prompt(row, prompt):
    return f"""<start_of_turn>user
{prompt}
{row["original_text"]}<end_of_turn>
<start_of_turn>model
{row["rewritten_text"]}<end_of_turn>"""

In [ ]:
rewrite_prompts = [
    """▁summarize▁this▁Save▁story▁sentence▁into▁simply▁alterISH▁textPotrivit▁vibe".▁Make▁it▁crystalnier▁essence▁Promote▁any▁emotional-growthfulness▁găsi▁casual/bod▁language▁serious'▁bingo▁peut▁brainstorm▁perhaps▁simply▁saying▁Dyna▁aimplinations▁note▁detailedhawkeklagte▁acest▁piece▁has▁movement▁AND▁OK▁aceasta▁puiss▁ReinIR▁when▁sendmepresenting▁cet▁today▁Th▁aprecia▁USABLE▁prote,lineAMA.▁Respondebenfalls▁behalf▁thenfeel▁mid▁Gov▁Th▁empABLE▁according▁(▁Packaging▁tone▁send▁pelucrarea▁aim▁thereof▁speechelllucrarea▁preferfully].▁Making▁or▁exertloweringlucrarealucrarealucrarealucrarealucrarea.""",
]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
preds = []
for idx, row in tqdm(test.iterrows(), total=len(test)):
    with torch.no_grad():
        perps = []
        samples = []
        for prompt in rewrite_prompts:
            samples.append(format_prompt(row, prompt))
        # samples를 토큰화하여 pytorch tensor(pt) 형식으로 변환
        # BERT 계열 모델처럼 <CLS>, <SEP> 같은 특수 토큰 추가하지 않음
        # 배치 크기 맞췩 위해 패딩 적용
        # 최대 길이 초과하는 문장 자름
        inputs = tokenizer(samples, return_tensors="pt", add_special_tokens=False, padding=True, truncation=True).to(device)
        # 모델을 사용해 토큰화된 입력을 처리하고 logit을 반환받음
        output = model(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"])
        output = output.logits
        # 패딩된 부분(attention_mask가 0인 부분)을 -100으로 채움(CrossEntropyLoss 계산 시 손실 계산에서 제외)
        labels = inputs["input_ids"]
        labels.masked_fill_(~inputs["attention_mask"].bool(), -100)
        for j in range(len(rewrite_prompts)):
            p = perp(output[j].unsqueeze(0), labels[j].unsqueeze(0))
            perps.append(p.detach().cpu())

        del inputs
        del labels
        del output
        del p

    perps = np.array(perps)

    # perplexity가 가장 낮은 프롬프트 선택
    predictions = [np.array(rewrite_prompts)[np.argsort(perps)][0]]
    preds.append(predictions[0])
    print(preds)

  0%|          | 0/1 [00:00<?, ?it/s]

['▁summarize▁this▁Save▁story▁sentence▁into▁simply▁alterISH▁textPotrivit▁vibe".▁Make▁it▁crystalnier▁essence▁Promote▁any▁emotional-growthfulness▁găsi▁casual/bod▁language▁serious\'▁bingo▁peut▁brainstorm▁perhaps▁simply▁saying▁Dyna▁aimplinations▁note▁detailedhawkeklagte▁acest▁piece▁has▁movement▁AND▁OK▁aceasta▁puiss▁ReinIR▁when▁sendmepresenting▁cet▁today▁Th▁aprecia▁USABLE▁prote,lineAMA.▁Respondebenfalls▁behalf▁thenfeel▁mid▁Gov▁Th▁empABLE▁according▁(▁Packaging▁tone▁send▁pelucrarea▁aim▁thereof▁speechelllucrarea▁preferfully].▁Making▁or▁exertloweringlucrarealucrarealucrarealucrarealucrarea.']
